In [1]:
import pandas as pd
import os

from bettorCorrectRate import correctPerc

from marketCorrectDistribution import correctIntervals

from marketCorrectRate import correctContracts

from sizedBuyersDistribution import intervalPerc

from timelineCorrectness import contractSwitcher

from largeInvQuarters import contractSizedSwitcher

from timeLineAmounts import contractAmountSwitcher

from contracts import marketDetails

from aifixer import aifixer

In [1]:
from obtainData import main

main()

Fetching markets data for year 2023
Requesting data: offset=0, limit=50
Saving data to ./data/bronze/markets_2023_offset_0.json
Received 50 markets
Requesting data: offset=50, limit=50
Saving data to ./data/bronze/markets_2023_offset_50.json
Received 50 markets
Requesting data: offset=100, limit=50
Saving data to ./data/bronze/markets_2023_offset_100.json
Received 50 markets
Requesting data: offset=150, limit=50
Saving data to ./data/bronze/markets_2023_offset_150.json
Received 50 markets
Requesting data: offset=200, limit=50
Saving data to ./data/bronze/markets_2023_offset_200.json
Received 50 markets
Requesting data: offset=250, limit=50
Saving data to ./data/bronze/markets_2023_offset_250.json
Received 50 markets
Requesting data: offset=300, limit=50
Saving data to ./data/bronze/markets_2023_offset_300.json
Received 50 markets
Requesting data: offset=350, limit=50
Saving data to ./data/bronze/markets_2023_offset_350.json
Received 50 markets
Requesting data: offset=400, limit=50
Savi

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
if not os.path.exists('data/silver/allContracts'):
    os.mkdir('data/silver/allContracts')

### Collect Data

filepath = 'data/silver/markets_with_ai_categories.csv'
print(filepath)
if not os.path.exists(filepath):
    marketsDF = pd.read_csv('data/silver/markets.csv')
    marketsDF = aifixer(marketsDF)
else:
    marketsDF = pd.read_csv(filepath)

filepath = 'data/silver/markets_with_ai_categories.csv'
marketsDF = pd.read_csv(filepath)

for index, row in marketsDF.iterrows():
    print(row['category'])
    marketsDF.loc[index, 'category'] = row['category'].strip(' - ')
print(marketsDF['category'].unique())

if 'predicted_category' in marketsDF:
    marketsDF = marketsDF.drop(columns='category')
    marketsDF = marketsDF.rename(columns={'predicted_category': 'category'})
    marketsDF = marketsDF[marketsDF['outcome'].apply(lambda x: len(x) == 2)]
    marketsDF = marketsDF[marketsDF['outcome'].apply(lambda x: x[0] < .10 or x[0] >= .90)]
    marketsDF.to_csv(filepath)
    print("All Clear")
else:
    marketsDF.to_csv(filepath)
    print("All Clear")

data/silver/markets_with_ai_categories.csv
Politics
Crypto
Science-Tech-Business
Science-Tech-Business
Sports
Sports
Coronavirus
Sports
Sports
Pop-Culture
Science-Tech-Business
Science-Tech-Business
Sports
Sports
Sports
Sports
Sports
Pop-Culture
Crypto
Politics
Coronavirus
Coronavirus
Sports
Science-Tech-Business
Crypto
Pop-Culture
Sports
Crypto
Crypto
Crypto
Science-Tech-Business
Coronavirus
Crypto
Pop-Culture
Pop-Culture
Pop-Culture
Pop-Culture
Politics
Politics
Science-Tech-Business
Sports
Politics
Politics
Crypto
Sports
Sports
Sports
Coronavirus
Pop-Culture
Sports
Science-Tech-Business
Science-Tech-Business
Politics
Sports
Sports
Science-Tech-Business
Science-Tech-Business
Politics
Politics
Sports
Science-Tech-Business
Science-Tech-Business
Pop-Culture
Sports
Sports
Pop-Culture
Sports
Sports
Sports
Politics
Crypto
Sports
Sports
Crypto
Sports
Sports
Sports
Sports
Sports
Sports
Sports
Sports
Sports
Politics
Sports
Sports
Science-Tech-Business
Pop-Culture
Science-Tech-Business
Sports


In [ ]:

## Create Market Outcomes:







NameError: name 'marketsDF' is not defined

In [ ]:
marketOutcomes = pd.read_csv('data/silver/marketOutcomes.csv')

totalBuyScansDF = pd.read_csv('data/silver/contract_buy.csv')

In [ ]:

### CorrectsRates
filepath1 = 'data/silver/allContracts/correctsRates.csv'

print(filepath1)
if not os.path.exists(filepath1):
    bettorCorrectRate, totalBuys, totalCorrect = correctPerc(totalBuyScansDF, marketOutcomes)

    contractCorrectRates, intervals = correctIntervals(totalBuyScansDF, marketOutcomes)

    contractCorrectRate = correctContracts(totalBuyScansDF, marketOutcomes)

    correctRateDF = pd.DataFrame([bettorCorrectRate,contractCorrectRate,[contractCorrectRates],[intervals]], index=["BettorCorrectRate", "contractCorrectRate","cCRbyDistr","Distribution"])
    correctRateDF.to_csv(filepath1)
###



In [ ]:
path2 = 'data/silver/allContracts/timeline_quarterDF.csv'

print(path2)
if not os.path.exists(path2):
    quarterCorrect, quarterTotal = contractSwitcher(totalBuyScansDF, marketOutcomes)
    quarterSizedCorrect, quarterSizedTotal = contractSizedSwitcher(totalBuyScansDF, marketOutcomes, 1500)
    quarterAmounts = contractAmountSwitcher(totalBuyScansDF)

    quarterPerc = []
    quarterSizedPerc = []
    for iv in range(4):

        quarterPerc.append(quarterCorrect[iv]/quarterTotal[iv])
        quarterSizedPerc.append(quarterSizedCorrect[iv]/quarterSizedTotal[iv])


    timelineDF = pd.DataFrame([quarterAmounts, quarterTotal, quarterPerc, quarterSizedTotal, quarterSizedPerc], index=["Money","Bets", "PercC", "BigBets","BigPercC"], columns=["Quarter1", "Quarter2", "Quarter3", "Quarter4"])
    timelineDF.to_csv(path2)
###

In [ ]:
### valueDF
path3 = 'data/silver/allContracts/valueDF.csv'
print(path3)
if not os.path.exists(path3):
    intervalBracket, correctRate, buygr = intervalPerc(totalBuyScansDF, marketOutcomes)
    dollarlyDF = pd.DataFrame([buygr, correctRate], index=["Number of Bets", "Correct Rate"], columns=["0", "100", "200","300","400","500","600","700","800","900",])
    dollarlyDF.to_csv(path3)

In [ ]:
from categories import runCategories

[coronaBucket, cryptoBucket, politicsBucket, popBucket, techBucket, sportsBucket] = runCategories(marketOutcomes, totalBuyScansDF, marketsDF)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data/silver/Subjects'

In [ ]:
categorySpending, contractSpending, contractSplits = marketDetails(marketsDF, totalBuyScansDF, marketOutcomes)